In [7]:
from mmdet.apis import init_detector, inference_detector
import mmcv
import cv2
import time
from mmengine.utils import track_iter_progress
import numpy as np
import sort

In [8]:
checkpoint = r"work_dirs\faster-rcnn_r50-tnr-pre_fpn_1x_coco_full_custom_br_hue\best_coco_Swimmer_HBB_precision_epoch_10.pth"
config = r"custom_configs\faster_rcnn\faster-rcnn_r50-tnr-pre_fpn_1x_coco_full_custom_br_hue.py"
video_path = r"C:\Users\tchar\Documents\GitHub\diving_analysis_bsc\flash_detection\PB_#3_5.avi"
name = "ouputs/faster_rcnn/"

In [9]:
# def extract_frame_debug(config, checkpoint, video_path, name):
#     model = init_detector(config, checkpoint, device="cpu")
#     cap = mmcv.VideoReader(video_path)
#     save_name = f"ouputs/faster_rcnn/{name}"
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter(
#     f"outputs/{save_name}.mp4", fourcc, cap.fps,
#     (cap.width, cap.height)
#     )
#     frame_count = 0 # To count total frames.
#     total_fps = 0 # To get the final frames per second.
#     for frame in track_iter_progress((cap, len(cap))):
#         frame_count += 1
#         start_time = time.time()# Forward pass start time.
#         result = inference_detector(model, frame)
#         end_time = time.time() # Forward pass end time.
#         # Get the fps.
#         fps = 1 / (end_time - start_time)
#         # Add fps to total fps.
#         total_fps += fps
#         #results[frame_count] = result
#         if result.pred_instances and result.pred_instances.scores is not None and len(result.pred_instances.scores) > 0 and max(result.pred_instances.scores) > 0.8:
#             # Get the index of the highest score
#             max_score_idx = np.argmax(result.pred_instances.scores)

#             # Get the bounding box corresponding to the highest score
#             highest_score_bbox = result.pred_instances.bboxes[max_score_idx]

#             # Extract x coordinate (rightmost x-coordinate)
#             x = highest_score_bbox[0]

#             threshold_x = 1024

#             print(f"Bounding box with highest score: {highest_score_bbox.tolist()}")
#             print(f"The x2 coordinate of the highest scoring bounding box is: {x.item()}")

#             if x <= threshold_x:
#                 print(f"The x-coordinate ({x.item():.2f}) is less than or equal to the threshold ({threshold_x}).")
#                 cv2.imwrite(f"outputs/faster_rcnn/frame{frame_count}.jpg", frame)
#                 return print(frame_count-1)
#                 # return frame_count
#             else:
#                 print(f"The x-coordinate ({x.item():.2f}) is greater than the threshold ({threshold_x}).")
#         else:
#             print("No predicted instances with scores found.")


#     out.release()
#     avg_fps = total_fps / frame_count
#     print(f"Average FPS: {avg_fps:.3f}")

In [10]:
# def extract_frame(config, checkpoint, video_path, save_path=False):
#     model = init_detector(config, checkpoint, device="cpu")
#     cap = mmcv.VideoReader(video_path)
#     frame_count = 0
#     for frame in cap:
#         frame_count += 1
#         result = inference_detector(model, frame)
#         if result.pred_instances and result.pred_instances.scores is not None and len(result.pred_instances.scores) > 0 and max(result.pred_instances.scores) > 0.8:
#             # Get the index of the highest score
#             max_score_idx = np.argmax(result.pred_instances.scores)

#             # Get the bounding box corresponding to the highest score
#             highest_score_bbox = result.pred_instances.bboxes[max_score_idx]

#             # Extract x coordinate (rightmost x-coordinate)
#             x = highest_score_bbox[0]

#             threshold_x = 1024

#             if x <= threshold_x:
#                 if save_path:
#                     cv2.imwrite(f"{save_path}/frame_{frame_count}.jpg", frame)
#                 cap.release()    
#                 return frame_count
        



In [11]:
def sort_extract_frame(config_path, checkpoint_path, video_path, pred_thresh, correction=0, save_path=False, sort_max_age=10, sort_min_hits=3, sort_iou_thresh=0.3):
    model = init_detector(config, checkpoint, device="cpu")
    cap = mmcv.VideoReader(video_path)
    save_name = f"ouputs/faster_rcnn/{name}"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(
    f"outputs/{save_name}.mp4", fourcc, cap.fps,
    (cap.width, cap.height)
    )
    frame_count = 0 # To count total frames.
    total_fps = 0 # To get the final frames per second.

    sort_tracker = sort.Sort(sort_max_age, sort_min_hits, sort_iou_thresh)

    for frame in track_iter_progress((cap, len(cap))):
        frame_count += 1
        start_time = time.time()# Forward pass start time.
        result = inference_detector(model, frame)
        end_time = time.time() # Forward pass end time.
        # Get the fps.
        fps = 1 / (end_time - start_time)
        # Add fps to total fps.
        total_fps += fps
        if result.pred_instances and result.pred_instances.scores is not None and len(result.pred_instances.scores) > 0 and max(result.pred_instances.scores) > 0.8:
            # Get the index of the highest score
            scores = result.pred_instances.scores
            max_score_idx = np.argmax(scores).item() # If using PyTorch, convert to Python int
            
            # Get the bounding box corresponding to the highest score
            highest_score_bbox_coords = result.pred_instances.bboxes[max_score_idx] # This is likely [x1, y1, x2, y2]
            highest_score = scores[max_score_idx].item() # Get the score as a scalar

            # Combine coords and score into the expected [x1, y1, x2, y2, score] format
            # and then wrap it in an array of arrays for Sort.update
            single_detection_array = np.array([[
                highest_score_bbox_coords[0].item(), # .item() to get scalar from tensor
                highest_score_bbox_coords[1].item(),
                highest_score_bbox_coords[2].item(),
                highest_score_bbox_coords[3].item(),
                highest_score # The score we got earlier
            ]])
           

            track_bb_ids = sort_tracker.update(single_detection_array)
            print(track_bb_ids)

            #Extract x coordinate (rightmost x-coordinate)
            if len(track_bb_ids) > 0:
                x = track_bb_ids[0][0]

                threshold_x = 1024

            
                if x <= threshold_x:
                    print(f"The x-coordinate ({x.item():.2f}) is less than or equal to the threshold ({threshold_x}).")
                    cv2.imwrite(f"outputs/faster_rcnn/frame{frame_count}.jpg", frame)
                    return print(frame_count-1)
                    # return frame_count
                else:
                    print(f"The x-coordinate ({x.item():.2f}) is greater than the threshold ({threshold_x}).")
            else:
                print("No predicted instances with scores found.")


    out.release()
    avg_fps = total_fps / frame_count
    print(f"Average FPS: {avg_fps:.3f}")

In [12]:
sort_extract_frame(config, checkpoint, video_path, name)

Loads checkpoint by local backend from path: work_dirs\faster-rcnn_r50-tnr-pre_fpn_1x_coco_full_custom_br_hue\best_coco_Swimmer_HBB_precision_epoch_10.pth
[                                                  ] 0/943, elapsed: 0s, ETA:

c:\Users\tchar\miniconda3\envs\mmdet\lib\site-packages\mmengine\runner\checkpoint.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, map

[                                 ] 6/943, 1.6 task/s, elapsed: 4s, ETA:   588s[]
No predicted instances with scores found.
[                                 ] 7/943, 1.6 task/s, elapsed: 4s, ETA:   588s[]
No predicted instances with scores found.
[                                 ] 8/943, 1.6 task/s, elapsed: 5s, ETA:   590s[[1723.09599832  163.93857009 1804.46503328  222.52827136    3.        ]]
The x-coordinate (1723.10) is greater than the threshold (1024).
[                                 ] 9/943, 1.6 task/s, elapsed: 6s, ETA:   593s[]
No predicted instances with scores found.
[                                ] 10/943, 1.6 task/s, elapsed: 6s, ETA:   597s[[1653.70353903  197.00810983 1761.43364692  270.18053377    3.        ]]
The x-coordinate (1653.70) is greater than the threshold (1024).
[                                ] 11/943, 1.6 task/s, elapsed: 7s, ETA:   596s[[1646.90896626  214.33211882 1745.73581824  285.5430933     3.        ]]
The x-coordinate (1646.91) is greater t

In [13]:
cv2.destroyAllWindows()